# Naolib Batch Analysis

This notebook performs batch analysis on the Naolib transportation data we've collected. We'll perform two batch analyses:
1. Average wait time by line and stop
2. Real-time prediction accuracy (comparing announced vs. actual wait times)

We'll use Spark for processing and Pandas + Seaborn for visualization.

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, explode, when, expr, to_timestamp, regexp_extract, count, avg, stddev
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

# Configure Spark
conf = SparkConf() \
    .setAppName('NaolibBatchAnalysis') \
    .setMaster('spark://spark:7077') \
    .set("spark.sql.shuffle.partitions", "10")

sc = SparkContext.getOrCreate(conf=conf)
sql_context = SQLContext(sc)

## 1. Load Data from Kafka into Spark

First, let's load the data we've collected from Kafka.

In [ ]:
# Define Kafka configurations
kafka_topic = "naolib_realtime"
kafka_server = "kafka:9092"

# Load the data from Kafka in batch mode
df_raw = sql_context.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

# Convert the binary value column to string
df_string = df_raw.selectExpr("CAST(value AS STRING)")

# Display a sample of the raw data
print("Sample of raw data:")
df_string.show(5, truncate=False)

## 2. Parse JSON and Extract Relevant Fields

Now, let's parse the JSON data and extract the fields we're interested in:

In [ ]:
# Parse JSON and extract arrivals array
df_expanded = df_string \
    .selectExpr(
        "json_tuple(value, 'timestamp', 'stop_code', 'stop_name', 'arrivals') as (timestamp, stop_code, stop_name, arrivals)"
    )

# Explode the arrivals array to get one row per arrival
df_arrivals = df_expanded \
    .withColumn("arrival", explode(expr("from_json(arrivals, 'array<struct<sens:string, terminus:string, temps:string, tempsReel:string, ligne:struct<numLigne:string>>>')")))
    .select(
        to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss").alias("timestamp"),
        col("stop_code"),
        col("stop_name"),
        col("arrival.sens").alias("direction"),
        col("arrival.terminus").alias("terminus"),
        col("arrival.temps").alias("wait_time_text"),
        col("arrival.tempsReel").alias("is_real_time"),
        col("arrival.ligne.numLigne").alias("line_number")
    )

# Convert wait time text to numeric minutes
df_clean = df_arrivals \
    .withColumn(
        "wait_time_minutes",
        when(col("wait_time_text") == "proche", 0)
        .when(col("wait_time_text").rlike("^\\d+$"), col("wait_time_text").cast("int"))
        .when(col("wait_time_text").rlike("^\\d+ min$"), regexp_extract(col("wait_time_text"), "(\\d+)", 1).cast("int"))
        .otherwise(None)  # For unexpected formats
    )

# Cache the dataframe for faster processing
df_clean.cache()

# Display schema and sample data
print("\nClean data schema:")
df_clean.printSchema()

print("\nSample of clean data:")
df_clean.show(5)

## 3. Batch Analysis 1: Average Wait Time by Line and Stop

Let's analyze the average wait times for each line at different stops:

In [ ]:
# Calculate average wait time by line and stop
avg_wait_by_line_stop = df_clean \
    .filter(col("wait_time_minutes").isNotNull()) \
    .groupBy("line_number", "stop_name") \
    .agg(
        avg("wait_time_minutes").alias("avg_wait_time"),
        stddev("wait_time_minutes").alias("stddev_wait_time"),
        count("*").alias("observation_count")
    ) \
    .filter(col("observation_count") > 5)  # Only include lines/stops with enough data
    
# Sort by average wait time (descending)
avg_wait_sorted = avg_wait_by_line_stop.orderBy(col("avg_wait_time").desc())

# Show the results
print("Average wait time by line and stop:")
avg_wait_sorted.show(10)

### Visualize Average Wait Times

In [ ]:
# Convert to Pandas for visualization
avg_wait_df = avg_wait_sorted.toPandas()

# Plot average wait times by line and stop
plt.figure(figsize=(12, 8))
sns.barplot(x="line_number", y="avg_wait_time", hue="stop_name", data=avg_wait_df)
plt.title("Average Wait Time by Line and Stop")
plt.xlabel("Line Number")
plt.ylabel("Average Wait Time (minutes)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create a heatmap for better visualization
plt.figure(figsize=(14, 10))
wait_time_pivot = avg_wait_df.pivot(index="stop_name", columns="line_number", values="avg_wait_time")
sns.heatmap(wait_time_pivot, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=.5)
plt.title("Average Wait Time Heatmap by Line and Stop")
plt.tight_layout()
plt.show()

## 4. Batch Analysis 2: Wait Time Distribution by Time of Day

Let's analyze how wait times vary throughout the day:

In [ ]:
from pyspark.sql.functions import hour

# Extract hour from timestamp
df_hourly = df_clean \
    .withColumn("hour_of_day", hour(col("timestamp"))) \
    .filter(col("wait_time_minutes").isNotNull())

# Calculate average wait time by hour of day
avg_wait_by_hour = df_hourly \
    .groupBy("hour_of_day") \
    .agg(
        avg("wait_time_minutes").alias("avg_wait_time"),
        stddev("wait_time_minutes").alias("stddev_wait_time"),
        count("*").alias("observation_count")
    ) \
    .orderBy("hour_of_day")

# Show the results
print("Average wait time by hour of day:")
avg_wait_by_hour.show(24)

### Visualize Wait Times by Hour

In [ ]:
# Convert to Pandas for visualization
hourly_wait_df = avg_wait_by_hour.toPandas()

# Plot average wait times by hour of day
plt.figure(figsize=(14, 6))
sns.lineplot(x="hour_of_day", y="avg_wait_time", data=hourly_wait_df, marker="o", linewidth=2)
plt.fill_between(
    hourly_wait_df["hour_of_day"],
    hourly_wait_df["avg_wait_time"] - hourly_wait_df["stddev_wait_time"],
    hourly_wait_df["avg_wait_time"] + hourly_wait_df["stddev_wait_time"],
    alpha=0.2
)
plt.title("Average Wait Time by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Average Wait Time (minutes)")
plt.xticks(range(0, 24))
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Also show observation count by hour
plt.figure(figsize=(14, 4))
sns.barplot(x="hour_of_day", y="observation_count", data=hourly_wait_df)
plt.title("Number of Observations by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Number of Observations")
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

## 5. Combined Analysis: Wait Time by Line and Hour

Let's combine the two analyses to get a more detailed picture:

In [ ]:
# Calculate average wait time by line and hour of day
avg_wait_by_line_hour = df_hourly \
    .groupBy("line_number", "hour_of_day") \
    .agg(
        avg("wait_time_minutes").alias("avg_wait_time"),
        count("*").alias("observation_count")
    ) \
    .filter(col("observation_count") > 3)  # Only include combinations with enough data
    
# Convert to Pandas for visualization
line_hour_wait_df = avg_wait_by_line_hour.toPandas()

# Select top 10 most frequent lines for visualization clarity
top_lines = df_clean.groupBy("line_number").count().orderBy(col("count").desc()).limit(10).select("line_number").toPandas()
top_line_list = top_lines["line_number"].tolist()

# Filter data for top lines
filtered_line_hour_df = line_hour_wait_df[line_hour_wait_df["line_number"].isin(top_line_list)]

# Plot heatmap of wait times by line and hour
plt.figure(figsize=(16, 10))
line_hour_pivot = filtered_line_hour_df.pivot(index="line_number", columns="hour_of_day", values="avg_wait_time")
sns.heatmap(line_hour_pivot, annot=True, fmt=".1f", cmap="YlOrRd", linewidths=.5)
plt.title("Average Wait Time by Line and Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Line Number")
plt.tight_layout()
plt.show()

# Create line plot for selected important lines
plt.figure(figsize=(14, 8))
for line in top_line_list[:5]:  # Just use top 5 lines for clarity
    line_data = filtered_line_hour_df[filtered_line_hour_df["line_number"] == line]
    if not line_data.empty:
        sns.lineplot(x="hour_of_day", y="avg_wait_time", data=line_data, marker="o", label=f"Line {line}")

plt.title("Average Wait Time Throughout the Day for Top Lines")
plt.xlabel("Hour of Day")
plt.ylabel("Average Wait Time (minutes)")
plt.xticks(range(0, 24))
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend(title="Line")
plt.tight_layout()
plt.show()

## 6. Analyzing Daily Patterns

Let's examine how wait times vary by day of the week:

In [ ]:
from pyspark.sql.functions import dayofweek, date_format

# Extract day of week from timestamp
df_daily = df_clean \
    .withColumn("day_of_week", date_format(col("timestamp"), "E")) \
    .filter(col("wait_time_minutes").isNotNull())

# Calculate average wait time by day of week
avg_wait_by_day = df_daily \
    .groupBy("day_of_week") \
    .agg(
        avg("wait_time_minutes").alias("avg_wait_time"),
        stddev("wait_time_minutes").alias("stddev_wait_time"),
        count("*").alias("observation_count")
    )

# Convert to Pandas for visualization
daily_wait_df = avg_wait_by_day.toPandas()

# Define correct day order
day_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
daily_wait_df["day_of_week"] = pd.Categorical(daily_wait_df["day_of_week"], categories=day_order, ordered=True)
daily_wait_df = daily_wait_df.sort_values("day_of_week")

# Plot average wait times by day of week
plt.figure(figsize=(12, 6))
sns.barplot(x="day_of_week", y="avg_wait_time", data=daily_wait_df)
plt.title("Average Wait Time by Day of Week")
plt.xlabel("Day of Week")
plt.ylabel("Average Wait Time (minutes)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

## 7. Terminus Analysis

Let's analyze which destinations have the longest wait times:

In [ ]:
# Calculate average wait time by terminus
avg_wait_by_terminus = df_clean \
    .filter(col("wait_time_minutes").isNotNull()) \
    .groupBy("terminus") \
    .agg(
        avg("wait_time_minutes").alias("avg_wait_time"),
        stddev("wait_time_minutes").alias("stddev_wait_time"),
        count("*").alias("observation_count")
    ) \
    .filter(col("observation_count") > 10)  # Only include terminus with enough data
    
# Sort by average wait time (descending)
terminus_wait_sorted = avg_wait_by_terminus.orderBy(col("avg_wait_time").desc())

# Show the results
print("Average wait time by terminus:")
terminus_wait_sorted.show(10)

# Convert to Pandas for visualization
terminus_wait_df = terminus_wait_sorted.toPandas()

# Plot top 15 terminus by average wait time
plt.figure(figsize=(14, 8))
sns.barplot(x="avg_wait_time", y="terminus", data=terminus_wait_df.head(15))
plt.title("Top 15 Destinations by Average Wait Time")
plt.xlabel("Average Wait Time (minutes)")
plt.ylabel("Destination")
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

## 8. Summary of Findings

Let's summarize our key findings from the batch analysis:

In [ ]:
# Overall statistics
overall_stats = df_clean \
    .filter(col("wait_time_minutes").isNotNull()) \
    .agg(
        count("*").alias("total_observations"),
        avg("wait_time_minutes").alias("overall_avg_wait_time"),
        stddev("wait_time_minutes").alias("overall_stddev_wait_time"),
        expr("percentile_approx(wait_time_minutes, 0.5)").alias("median_wait_time"),
        expr("min(wait_time_minutes)").alias("min_wait_time"),
        expr("max(wait_time_minutes)").alias("max_wait_time")
    )

# Show overall statistics
print("Overall statistics:")
overall_stats.show()

# Get top 5 lines with longest average wait times
top_wait_lines = df_clean \
    .filter(col("wait_time_minutes").isNotNull()) \
    .groupBy("line_number") \
    .agg(avg("wait_time_minutes").alias("avg_wait_time"), count("*").alias("count")) \
    .filter(col("count") > 10) \
    .orderBy(col("avg_wait_time").desc()) \
    .limit(5)

print("\nTop 5 lines with longest average wait times:")
top_wait_lines.show()

# Get top 5 stops with longest average wait times
top_wait_stops = df_clean \
    .filter(col("wait_time_minutes").isNotNull()) \
    .groupBy("stop_name") \
    .agg(avg("wait_time_minutes").alias("avg_wait_time"), count("*").alias("count")) \
    .filter(col("count") > 10) \
    .orderBy(col("avg_wait_time").desc()) \
    .limit(5)

print("\nTop 5 stops with longest average wait times:")
top_wait_stops.show()

# Create a textual summary of findings
print("\n=== SUMMARY OF FINDINGS ===")
print("Based on our analysis of Naolib transportation data in Nantes, we found:")
print("1. The overall average wait time across all lines and stops is approximately X minutes.")
print("2. Wait times tend to be longest during [morning/evening] peak hours and lowest during [off-peak].")
print("3. Lines [X, Y, Z] consistently have the longest wait times across all stops.")
print("4. The stops with the longest average wait times are [A, B, C].")
print("5. Wait times are generally [higher/lower] on weekends compared to weekdays.")
print("6. Destinations to [X, Y, Z] have the longest wait times on average.")